# Lab 11: AI Guardrails

## Initial Setup of Guardrails

In [1]:
# Download guardrails
# install guardrails 
! pip install guardrails-ai

### CREATE GUARDRAILS APIKEY

In [2]:
# Generate an APIKEY 
# https://hub.guardrailsai.com/keys

## Load Environment Table
from dotenv import load_dotenv
load_dotenv(override=True)  # take environment variables

True

In [ ]:
# Configure the guardrails 
# !guardrails configure
# 

In [3]:
# Load large language model 
from langchain.chat_models import init_chat_model
llm = init_chat_model("gpt-4.1-nano", model_provider="openai")

## Setup Finished
---
## Testing our First Validator: ReGex Match

In [4]:
## Install the validator
from guardrails import Guard, install

install("hub://guardrails/regex_match")

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>
/Users/hizkiafebianto/Documents/GitHub/revou-gen-ai-tutorial/venv/lib/python3.11/site-packages/guardrails/hub/install.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Installing hub://guardrails/regex_match...

✅Successfully installed guardrails/regex_match version 0.0.0!

<module 'guardrails_grhub_regex_match' from '/Users/hizkiafebianto/Documents/GitHub/revou-gen-ai-tutorial/venv/lib/python3.11/site-packages/guardrails_grhub_regex_match/__init__.py'>

In [5]:
from guardrails.hub import RegexMatch
from guardrails import Guard 

# Initialize the Guard 
guard = Guard().use(
    RegexMatch(regex="^[A-Z][a-z]*$")
)

In [ ]:
# guard.parse("Dexa")
guard.parse("Dexa Medica") # --> Error validation. Let's catch the error. 

ValidationError: Validation failed for field with errors: Result must match ^[A-Z][a-z]*$

In [ ]:
# Catching the Validation Error
from guardrails.errors import ValidationError

try:
    # Failed validation
    guard.parse("Dexa Medica")
except (ValidationError) as e:
    print(e)
    # Add error handling function

Validation failed for field with errors: Result must match ^[A-Z][a-z]*$


---
## Combining multiple Validators

In [10]:
# Installing the validator
install("hub://guardrails/valid_length")

Installing hub://guardrails/valid_length...

✅Successfully installed guardrails/valid_length version 0.0.0!

<module 'guardrails_grhub_valid_length' from '/Users/hizkiafebianto/Documents/GitHub/revou-gen-ai-tutorial/venv/lib/python3.11/site-packages/guardrails_grhub_valid_length/__init__.py'>

In [11]:
from guardrails.hub import ValidLength, RegexMatch
from guardrails import Guard

guard = Guard().use_many(
    RegexMatch(regex="^[A-Z][a-z]*$"),
    ValidLength(min=1, max=12),
    # SecretsPresent(on_fail="fix")
)

In [14]:
# print(guard.parse("Caesar").validation_passed)  # Guardrail Passes
#print(
#    guard.parse("Caesar Salad")
#    .validation_passed
#)  # Guardrail Fails due to regex match
print(
    guard.parse("Caesarisagreatleader")
    .validation_passed
)  # Guardrail Fails due to length

ValidationError: Validation failed for field with errors: Value has length greater than 12. Please return a shorter output, that is shorter than 12 characters.

---
## on_fail Actions 

In [15]:
# setup, run imports
try:
    from guardrails.hub import DetectPII
except ImportError:
    install("hub://guardrails/detect_pii")
    from guardrails.hub import DetectPII

In [16]:
# Raising Exception
guard = Guard().use(
    DetectPII(pii_entities="pii", on_fail="exception")
)

try:
    guard.validate("Hello, my name is John Doe and my email is john.doe@example.com")
except Exception as e:
    print("output validation failed")
    print(e)

output validation failed
Validation failed for field with errors: The following text in your response contains PII:
Hello, my name is John Doe and my email is john.doe@example.com


In [17]:
# Doing Nothing
guard = Guard().use(
    DetectPII(pii_entities="pii", on_fail="noop")
)

guard.validate("Hello, my name is John Doe and my email is john.doe@example.com")

ValidationOutcome(call_id='14362570304', raw_llm_output='Hello, my name is John Doe and my email is john.doe@example.com', validation_summaries=[ValidationSummary(validator_name='DetectPII', validator_status='fail', property_path='$', failure_reason='The following text in your response contains PII:\nHello, my name is John Doe and my email is john.doe@example.com', error_spans=[ErrorSpan(start=18, end=26, reason='PII detected in John Doe')])], validated_output='Hello, my name is John Doe and my email is john.doe@example.com', reask=None, validation_passed=False, error=None)

In [18]:
# Doing Nothing
guard = Guard().use(
    DetectPII(pii_entities="pii", on_fail="fix")
)

validation_result = guard.validate("Hello, my name is John Doe and my email is john.doe@example.com")

In [19]:
validation_result.validated_output

'Hello, my name is <PERSON> and my email is <EMAIL_ADDRESS>'

In [21]:
messages=[{
        "role": "user",
        "content": "Make up a fake person and email address",
    }]
llm.invoke(messages)

AIMessage(content="Sure! Here's a fake person and email address for you:\n\nName: Alex Morgan  \nEmail: alex.morgan@example.com", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 15, 'total_tokens': 39, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_f12167b370', 'id': 'chatcmpl-BcpVcDHecK1C8su0MDMtIuYk0mw8n', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--04709f5a-4c1c-4ce4-b1aa-cbc15b138f99-0', usage_metadata={'input_tokens': 15, 'output_tokens': 24, 'total_tokens': 39, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [22]:
guard = Guard().use(
    DetectPII(pii_entities="pii", on_fail="reask"),
)

res = guard(
    messages=[{
        "role": "user",
        "content": "Make up a fake person and email address",
    }],
    model='gpt-4o-mini',
    num_reasks=1
)

print("Validated output: ", res.validated_output)
print("Number of reasks: ", len(guard.history.last.iterations) - 1)

/Users/hizkiafebianto/Documents/GitHub/revou-gen-ai-tutorial/venv/lib/python3.11/site-packages/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


Validated output:  Sure! Here's a fictional person without any personal identifiable information:

**Name:** <PERSON>  
**Email:** <EMAIL_ADDRESS>  

Feel free to use this for any creative purposes!
Number of reasks:  1


---
# Handling Structured Data 

In [86]:
install("hub://guardrails/lowercase")

Installing hub://guardrails/lowercase...

✅Successfully installed guardrails/lowercase version 0.0.0!

<module 'guardrails_grhub_lowercase' from '/Users/hizkiafebianto/Documents/GitHub/revou-gen-ai-tutorial/venv/lib/python3.11/site-packages/guardrails_grhub_lowercase/__init__.py'>

In [87]:
install("hub://guardrails/valid_range")

Installing hub://guardrails/valid_range...

✅Successfully installed guardrails/valid_range version 0.0.0!

<module 'guardrails_grhub_valid_range' from '/Users/hizkiafebianto/Documents/GitHub/revou-gen-ai-tutorial/venv/lib/python3.11/site-packages/guardrails_grhub_valid_range/__init__.py'>

In [90]:
from guardrails.hub import LowerCase, ValidRange

prompt = """
Given the following fast food order, please provide a summary of the orders.
${order}
${gr.complete_xml_suffix_v2}
"""

order = """I want a BURGER with two LARGE FRIES and a coke zero."""

# MinimumOneRange is a hypothetical custom validator that an integer > 0 is supplied
class Lineitem(BaseModel):
    item: str = Field(description="The name of the item being ordered.", validators=[LowerCase(on_fail="fix")])
    quantity: int = Field(description="The quantity of the item being ordered", validators=[ValidRange(min=1, max=10, on_fail="fix")])

guard = Guard.from_pydantic(output_class=List[Lineitem])

response = guard(
    model="gpt-4.1",
    messages=[{
        "role": "system",
        "content": "You are a helpful assistant."
    },{
        "role": "user",
        "content": prompt
    }],
    prompt_params={"order": order},
)

print(response.validated_output)

# [{'item': 'burger', 'quantity': 1},
#  {'item': 'fries', 'quantity': 2},
#  {'item': 'coke zero', 'quantity': 1}]

[{'item': 'burger', 'quantity': 1}, {'item': 'large fries', 'quantity': 2}, {'item': 'coke zero', 'quantity': 1}]


In [91]:
response.validation_passed

True

In [92]:
response.raw_llm_output

'[\n  {"item": "burger", "quantity": 1},\n  {"item": "large fries", "quantity": 2},\n  {"item": "coke zero", "quantity": 1}\n]'

In [93]:
response.validated_output

[{'item': 'burger', 'quantity': 1},
 {'item': 'large fries', 'quantity': 2},
 {'item': 'coke zero', 'quantity': 1}]

# END